I want to see whether the introduction of mandatory emissions checks on cars every two years since 1983 correlates with an increase in air quality.

According to the "State of the Air" report 2020 by the American Lung Association there is a general decreasing trend in emissions since the 1970's in the US.

According to https://www.cabq.gov/airquality/trends there is also a downward trend between 2000 - 2015 in:
1. Ozone (ppm)
2. Carbon Monoxide (ppm)
3. Nitrogen Dioxide (ppb)
concentration in parts per million or parts per billion in Albuqurque.

Levels of particulate matter (both 10, 2.5 microns)and Sulfur Dioxide, however appear to be stable over the same time period.

In [5]:
import pandas as pd

In [7]:
#Change in Ozone levels (Parts per million)
levelOz_2000 = 0.075
levelOz_2015 = 0.066
change_ozone = round(levelOz_2000 - levelOz_2015, 4)

#Change in Carbon Monoxide levels (Parts per million)
levelCO_2000 = 3.8
levelCO_2015 = 1.4
change_CO = round(levelCO_2000 - levelCO_2015, 4)

#Change in Nitrogen Dioxide levels (Parts per billion)
levelNO_2000 = 65
levelNO_2015 = 45
change_NO = levelNO_2000 - levelNO_2015
#convert to ppm
change_NO = change_NO/1000

In [14]:
#Percent change Ozone
percent_ozone = round(change_ozone/levelOz_2000 * 100, 4)
percent_CO = round(change_CO/levelCO_2000 * 100, 4)
precent_NO = round(change_NO/levelNO_2000 * 100, 4)

In [17]:
data = {"change_ppm": [change_ozone, change_CO, change_NO], "percent_change": [percent_ozone, percent_CO, precent_NO]}
df_change = pd.DataFrame(data = data, index = ["Ozone", "Carbon_Monoxide", "Nitrogen_Dioxide"])
print("Change in three air pollutants from 2000 to 2015 in Albuquerque")
df_change

Change in three air pollutants from 2000 to 2015 in Albuquerque


,change_ppm,percent_change
Ozone,0.009,12.0000
Carbon_Monoxide,2.400,63.1579
Nitrogen_Dioxide,0.020,0.0308
